In [2]:
#LIBRERIAS NECESARIAS:
#Para utilizar API
import requests
#Para realizar la estructura tabular
import pandas as pd
#Para rellenar vacíos
import numpy as np
import json

#ETL:

#para normalizar strings
from unicodedata import normalize
#para normalizar incluyendo la ñ
import re 
#para normalizar fechas
import datetime
#para obtener fechas
import time
#hacer los calendarios de iteración
from dateutil.rrule import rrule, DAILY , MONTHLY

import mysql.connector
from sqlalchemy import create_engine

#para normalizar fechas
from datetime import datetime
#para obtener fechas
import time

dic={}

<span style="font-size: 20px; color:cyan; font-weight: bold;"> - Conexión al  servidos RDS</span>

In [3]:
host = "database-alertas-sismicas.crcnepco0igw.us-east-1.rds.amazonaws.com"
user = "admin"
password = "admin4168"
database = "alertasSismicas"


In [9]:
con=create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

In [21]:
connection = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)


<span style="font-size: 20px; color:cyan; font-weight: bold;"> - Creation of data tables.</span>

In [7]:
data_countries = 'DROP TABLE IF EXISTS COUNTRIES CASCADE; CREATE TABLE COUNTRIES (idcountry INTEGER AUTO_INCREMENT PRIMARY KEY NOT NULL ,country text);'
data_jap = 'DROP TABLE IF EXISTS JAPAN CASCADE; CREATE TABLE JAPAN (idsismo SERIAL PRIMARY KEY NOT NULL,idcountry INTEGER,foreign key (idcountry) references COUNTRIES(idcountry),mag float8, place text,time timestamp,tsunami text,lng float8, lat float8,depth float8, danger smallint);'
data_usa = 'DROP TABLE IF EXISTS USA CASCADE; CREATE TABLE USA (idsismo SERIAL PRIMARY KEY NOT NULL,idcountry INTEGER,foreign key (idcountry) references COUNTRIES(idcountry),mag float8, place text,time timestamp,tsunami text,lng float8, lat float8,depth float8, danger smallint);'
data_chile = 'DROP TABLE IF EXISTS CHILE CASCADE; CREATE TABLE CHILE (idsismo SERIAL PRIMARY KEY NOT NULL,idcountry INTEGER,foreign key (idcountry) references COUNTRIES(idcountry),mag float8, place text,time timestamp,tsunami text,lng float8, lat float8,depth float8, danger smallint);'
data_fact = 'DROP TABLE IF EXISTS FACTS CASCADE; CREATE TABLE FACTS (idsismo SERIAL PRIMARY KEY NOT NULL,idcountry INTEGER,foreign key (idcountry) references COUNTRIES(idcountry),mag float8, place text,time timestamp,tsunami text,lng float8, lat float8,depth float8, danger smallint);'
data_tsu = 'DROP TABLE IF EXISTS TSUNAMIS CASCADE; CREATE TABLE TSUNAMIS (id SERIAL PRIMARY KEY NOT NULL ,idcountry INTEGER,foreign key (idcountry) references COUNTRIES(idcountry),mxwatheight float8, place text, time timestamp, mag float8, lng float8, lat float8, depth float8);'

In [4]:
data_test = 'DROP TABLE IF EXISTS SISMOTEST CASCADE; CREATE TABLE SISMOTEST (idsismo SERIAL PRIMARY KEY NOT NULL,idcountry INTEGER,foreign key (idcountry) references COUNTRIES(idcountry),mag float8, place text,time timestamp,tsunami text,lng float8, lat float8,depth float8, danger smallint);'

In [20]:
data_alarm = 'DROP TABLE IF EXISTS ALARMS CASCADE;CREATE TABLE ALARMS (idsismo SERIAL PRIMARY KEY NOT NULL,idcountry INTEGER,foreign key (idcountry) references COUNTRIES(idcountry),mag float8, place text,time timestamp,tsunami text,lng float8, lat float8,depth float8, danger smallint);'

In [11]:
cursor=connection.cursor()
cursor.execute(data_countries)
cursor.close()

True

In [13]:
cursor=connection.cursor()
cursor.execute(data_jap)
cursor.close()

True

In [15]:
cursor=connection.cursor()
cursor.execute(data_chile)
cursor.close()

True

In [17]:
cursor=connection.cursor()
cursor.execute(data_usa)
cursor.close()

True

In [19]:
cursor=connection.cursor()
cursor.execute(data_fact)
cursor.close()

True

In [21]:
cursor=connection.cursor()
cursor.execute(data_tsu)
cursor.close()

True

In [6]:
cursor=connection.cursor()
cursor.execute(data_test)
cursor.close()

True

In [22]:
cursor=connection.cursor()
cursor.execute(data_alarm)
cursor.close()

True

<span style="font-size: 20px; color:cyan; font-weight: bold;"> - Development of the Countries table. </span>

In [22]:
def COUNTRIES(country):

    global dic
    if country in dic:
        #df.to_sql(name='pais',con=cone, if_exists='replace', index=False)
        return(dic[country])
    else:
        dic[country]=len(dic)+1
        dic_pais={'COUNTRY':[country]}
        df=pd.DataFrame(dic_pais)
        df.to_sql(name='COUNTRIES',con=con, if_exists='append', index=False)
        
        return(dic[country])  


In [23]:
COUNTRIES("USA")
COUNTRIES("CHILE")
COUNTRIES("JAPAN")

3

<span style="font-size: 20px; color:cyan; font-weight: bold;"> - Development of the Historic table. </span>

In [24]:
url_japan = f'https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=2010-01-01%2000:00:00&endtime=2023-12-31%2023:59:59&minlatitude=27.000000&maxlatitude=44.000000&minlongitude=132.780000&maxlongitude=145.530000&minmagnitude=3&orderby=time-asc&jsonerror=true'
dt_in_jap=pd.read_csv(url_japan,delimiter=',')
url_chile = f'https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=2010-01-01%2000:00:00&endtime=2023-12-31%2023:59:59&minlatitude=-56.800000&maxlatitude=-19.000000&minlongitude=-79.000000&maxlongitude=-69.&minmagnitude=3&orderby=time-asc&jsonerror=true'
dt_in_ch=pd.read_csv(url_chile,delimiter=",")
url_usa = f'https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=2010-01-01%2000:00:00&endtime=2023-12-31%2023:59:59&maxlatitude=50&minlatitude=24.6&maxlongitude=-65&minlongitude=-125&minmagnitude=3&orderby=time-asc&jsonerror=true'
dt_in_usa=pd.read_csv(url_usa,delimiter=',')

In [25]:
def transformar_fecha(dato):
    fecha_sin_tz = dato.replace("T", " ").replace("Z", "")
    fecha_objeto = datetime.strptime(fecha_sin_tz, "%Y-%m-%d %H:%M:%S.%f")
    fecha_formateada = fecha_objeto.strftime("%Y/%m/%d %H:%M:%S")
    return fecha_formateada

In [26]:
def generat_dat (df,pais:str):
    df["place"]=df["place"].fillna("no data")
    df=df.fillna(0)
    df=df.drop_duplicates()
    df=df[["mag","place","time","longitude","latitude","depth"]]
    df['time'] = df['time'].apply(lambda x: transformar_fecha(x))
    df.rename(columns={'latitude':'lat', 'longitude':'lng' }, inplace=True)
    df["idcountry"]=COUNTRIES(pais)
    df["tsunami"]=0
    df["danger"]="1"
    df=df[["idcountry","mag","place","time","tsunami","lng","lat","depth","danger"]]
    
       
    return df
    

In [27]:
dt_his_usa=generat_dat(dt_in_usa,"USA")
dt_his_chil=generat_dat(dt_in_ch,"CHILE")
dt_his_jap=generat_dat(dt_in_jap,"JAPAN")

In [28]:
dt_his_usa.to_sql(name='USA',con=con, if_exists='append', index=False)
dt_his_usa.to_sql(name='FACTS',con=con, if_exists='append', index=False)

dt_his_chil.to_sql(name='CHILE',con=con, if_exists='append', index=False)
dt_his_chil.to_sql(name='FACTS',con=con, if_exists='append', index=False)

dt_his_jap.to_sql(name='JAPAN',con=con, if_exists='append', index=False)
dt_his_jap.to_sql(name='FACTS',con=con, if_exists='append', index=False)

14715

<span style="font-size: 20px; color:cyan; font-weight: bold;"> - Generate data from tsunamis. </span>

In [29]:
url = f'https://www.ngdc.noaa.gov/hazel/hazard-service/api/v1/tsunamis/events?country=USA&maxYear=2023&minYear=2010'
dt_usa_prev_Mar = pd.DataFrame(requests.get(url).json()['items'])

url = f'https://www.ngdc.noaa.gov/hazel/hazard-service/api/v1/tsunamis/events?country=JAPAN&maxYear=2023&minYear=2010'
dt_jap_prev_Mar = pd.DataFrame(requests.get(url).json()['items'])

url = f'https://www.ngdc.noaa.gov/hazel/hazard-service/api/v1/tsunamis/events?country=CHILE&maxYear=2023&minYear=2010'
dt_chi_prev_Mar = pd.DataFrame(requests.get(url).json()['items'])

In [30]:
def mar(df):
    
    df=df.fillna(0)
    df['time'] = pd.to_datetime(df[['year','month','day','hour','minute','second']],format='%Y/%m/%d %H:%M:%S')
    df['time'] =df['time'].apply(lambda x: x.strftime('%Y/%m/%d %H:%M:%S'))
    df.rename(columns={'eqMagnitude':'mag', 'eqDepth':'depth', 'latitude':'lat', 'longitude':'lng',
    'locationName':'place', 'maxWaterHeight':'mxwatheight'}, inplace=True)


    df['mxwatheight']  = df.mxwatheight.astype('float64')
    df.mag=df.mag.astype('float64')
    df.lng=df.lng.astype('float64')
    df.lat=df.lat.astype('float64')
    df.depth=df.depth.astype('float64')

    a=df["country"][1]
    df["idcountry"]=COUNTRIES(a)
    
    # Filtramos el dataframe con los datos relevantes.
    df = df[['idcountry','mxwatheight','place', 'time', 'mag', 'lng','lat', 'depth' ]]
    
    return (df)    
    

In [31]:
dt_usa_Mar=mar(dt_usa_prev_Mar)
dt_chi_Mar=mar(dt_chi_prev_Mar)
dt_jap_Mar=mar(dt_jap_prev_Mar)

In [32]:
dt_usa_Mar.to_sql(name='TSUNAMIS',con=con, if_exists='append', index=False)
dt_chi_Mar.to_sql(name='TSUNAMIS',con=con, if_exists='append', index=False)
dt_jap_Mar.to_sql(name='TSUNAMIS',con=con, if_exists='append', index=False)

21

<span style="font-size: 20px; color:cyan; font-weight: bold;"> - Generate datatest from Sismos. </span>

In [16]:
def sismos_Noaa():

    import requests
    import pandas as pd
    from IPython.display import display
    countries = ['USA', 'CHILE', 'JAPAN']
    dt=pd.DataFrame(index=['idpais','mag','place', 'time','tsunami', 'lng','lat', 'deepth'])
    for country in countries:
        if country=="USA": z=1
        elif country=="CHILE": z=2
        elif country=="JAPAN": z=3
        # Obtenemos información a partir de la API
        url = f'https://www.ngdc.noaa.gov/hazel/hazard-service/api/v1/earthquakes?country={country}&maxYear=2023&minYear=2000'
        df = pd.DataFrame(requests.get(url).json()['items'])

        # Creamos la columna con la fecha del sismo
        df['time'] = pd.to_datetime(df[['year','month','day','hour','minute','second']])

        # Renombramos los features para normalizar con los otros datos recolectados
        df.rename(columns={'eqMagnitude':'mag', 'eqDepth':'depth', 'latitude':'lat', 'longitude':'lng', 'locationName':'place', 'country':'pais'}, inplace=True)

        # Creamos features adicionales para normalizar los datos.
        df['url'] = url
        df['title'] = None
        df['tsunami'] = df.apply(lambda x: 1 if pd.isnull(x.tsunamiEventId) != True else 0, axis=1)
        df['idcountry']  = z
        df['place']  = df.apply(lambda x: x.place.lower(),axis=1)
        
        # Filtramos el dataframe con los datos relevantes.
        df = df[['idcountry','mag','place', 'time','tsunami', 'lng','lat', 'depth']]
        df['danger']=3
        dt=pd.concat([dt,df],ignore_index=True)
        dt= dt.dropna(how='all')
        dt=dt.sort_values("time", ascending=False)

    return(dt)

In [17]:
data_test_sismo=sismos_Noaa()

In [18]:
data_test_sismo.to_sql(name='SISMOTEST',con=con, if_exists='append', index=False)

162